In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
# !pip install kaggle

In [ ]:
# root_path = '/content/gdrive/MyDrive/kaggleProject'  #change dir to your project folder

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
!pwd

/content


In [ ]:
!mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
!cp /content/gdrive/MyDrive/kaggleProject/kaggle.json ~/.kaggle/ # for use on tonmoy's gdrive
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json 

kaggle.json


In [ ]:
# !cp kaggle-tonmoy.json /content/gdrive/MyDrive/kaggleProject

In [ ]:
!kaggle competitions download -c dlsprint
# !kaggle competitions download -c dlsprint -p /content/gdrive/MyDrive/kaggleProject/dlsprint

100% 7.42G/7.43G [00:55<00:00, 199MB/s]
100% 7.43G/7.43G [00:55<00:00, 143MB/s]


In [ ]:
# import os
# os.chdir('gdrive/My Drive/kaggle/dlsprint')  #change dir
# !mkdir train  #create a directory named train/
# !mkdir test  #create a directory named test/
# !unzip -q train.zip -d train/  #unzip data in train/
# !unzip -q test.zip -d test/  #unzip data in test/
# !unzip sample_submission.csv.zip
# !unzip train_labels.csv.zip

In [ ]:
import zipfile
extract_path = '/content/input/dlsprint'
with zipfile.ZipFile('/content/dlsprint.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import unicodedata
text="বায়ান্ন"
new_text = unicodedata.normalize("NFKC", text)
print("text:",text)
print("new text:",new_text)
print("are they same?:",text==new_text)

In [ ]:
print("first one:",[f for f in 'কেন্দ্রীয়'])
print("second one:",[f for f in 'কেন্দ্রীয়'])

In [ ]:
print("first one:",[f for f in text])
print("second one:",[f for f in new_text])

## Importhing Necessary Libraries for EDA

In [ ]:
import os

import numpy as np
import pandas as pd

import librosa
import librosa.display

import IPython.display as ipd # for displaying the play audio buttom

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


## Reading the .csv files

In [ ]:
extract_path = '/content/input/dlsprint'
os.path.join(extract_path,)
train_df = pd.read_csv(os.path.join(extract_path,'train.csv'))
val_df = pd.read_csv(os.path.join(extract_path,'validation.csv'))
sample_submissin_df = pd.read_csv(os.path.join(extract_path,'sample_submission.csv'))

## Take a look at the .csv files

In [ ]:
train_df.head(n = 3)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale
0,6c274d3678353b62f5d3465c771e4a8c5cd917186a5323...,common_voice_bn_30991326.mp3,বাবা সত্যেন ঘোষ।,1.0,1.0,NaN,NaN,NaN,bn
1,3991c2cfe1be64e77f2f6bdaeaae601219af906c1295cd...,common_voice_bn_30991371.mp3,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ...,1.0,0.0,NaN,NaN,NaN,bn
2,3991c2cfe1be64e77f2f6bdaeaae601219af906c1295cd...,common_voice_bn_30991410.mp3,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর...,0.0,0.0,NaN,NaN,NaN,bn


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206950 entries, 0 to 206949
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   client_id   206950 non-null  object 
 1   path        206950 non-null  object 
 2   sentence    206950 non-null  object 
 3   up_votes    206950 non-null  float64
 4   down_votes  206950 non-null  float64
 5   age         60309 non-null   object 
 6   gender      61196 non-null   object 
 7   accents     11088 non-null   object 
 8   locale      206950 non-null  object 
dtypes: float64(2), object(7)
memory usage: 14.2+ MB


In [ ]:
train_df.isna().sum()

client_id          0
path               0
sentence           0
up_votes           0
down_votes         0
age           146641
gender        145754
accents       195862
locale             0
dtype: int64

In [ ]:
train_df['down_votes'].unique()

array([ 1.,  0.,  5.,  2.,  3.,  6.,  4.,  7., 14., 10.,  9., 13., 16.,
        8., 21., 17., 12., 11., 22., 24., 30., 18., 48., 20.])

In [ ]:
train_df['up_votes'].unique()

array([  1.,   0.,   6.,   2.,   4.,   3.,  16.,   7.,   8.,   5.,  14.,
        11.,  15.,  10.,  12.,  13.,  17.,   9.,  18.,  20.,  30.,  26.,
        24.,  22.,  32.,  28.,  29.,  21.,  53.,  34.,  23.,  38.,  35.,
        19.,  36.,  27.,  40.,  25.,  37.,  31.,  45.,  39.,  50.,  33.,
        47.,  42.,  46.,  66.,  59.,  78.,  56.,  44., 108.,  88.,  55.,
        43.,  41.,  60.,  92., 124.,  51.,  54.,  52.,  48.,  68.,  62.,
       106.,  58.,  80.,  74.,  70.,  84.,  94.])

Higher the upvotes more accurate the text labels

Samples with higher number of downvote than upper are considered as incorrect.

## Let't check how label them

In [ ]:
train_df['locale'].unique()

array(['bn'], dtype=object)

So all the audio files are labled by native bengalies

*Columns named 'age', 'gender', 'accents', 'locale' would be removed as they didn't carry important information for the project.*

In [ ]:
train_df = train_df.drop(['age', 'gender', 'accents', 'locale'],axis = 1)
val_df = val_df.drop(['age', 'gender', 'accents', 'locale'],axis = 1)

In [ ]:
# librosa.load() takes the path and returns a numpy array and the sample rate of the audio.
# extract_path = '/content/input/dlsprint'
AUDIO_PATH = extract_path+'/train_files/common_voice_bn_30614352.mp3'
audio, sr = librosa.load(AUDIO_PATH)

print('Shape of the audio: ', audio.shape)
print('Sample rate of the audio: ', sr)
ipd.display(ipd.Audio(data=audio, rate=sr))
print('Audio Label :',train_df[train_df['path'] == os.path.split(AUDIO_PATH)[1]].sentence)

Shape of the audio:  (107163,)
Sample rate of the audio:  22050


## All the audio files are in .mp3 format we need to convert them into .wav format for faster processing.

In [ ]:
!pip install pydub

Audio Label : 195102    বাকি অর্ধেক ডানা সাদা বর্ণের।
Name: sentence, dtype: object
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import skimage.io

import time

from tqdm.notebook import tqdm
tqdm.pandas()

import shutil

from pydub import AudioSegment

from joblib import Parallel, delayed

import librosa

import io
import soundfile as sf

In [ ]:
# extract_path = '/content/input/dlsprint'
TRAIN_PATH = os.path.join(extract_path,'train_files') 
TEST_PATH = os.path.join(extract_path,'test_files')
VALIDATION_PATH = os.path.join(extract_path,'validation_files')


In [ ]:
#---------------
# data filtering
#---------------
def filter_votes(x):
    up=x["up_votes"]
    down=x["down_votes"]
    # if up-down<=0:
    if up<down:
        return None
    # elif up==0:
    #     return None
    else:
        return up

In [ ]:
print("Total Data before filtering:",len(train_df))
train_df["up_votes"]=train_df.progress_apply(lambda x:filter_votes(x),axis=1)
train_df.dropna(subset = ['up_votes'],inplace=True)
print("Total Data after filtering:",len(train_df))
train_audio_files=train_df["path"].tolist()
train_df.to_csv('new_train.csv')

Total Data before filtering: 206950


  0%|          | 0/206950 [00:00<?, ?it/s]

Total Data after filtering: 201414


## Creating a list of trian files

In [ ]:
# list to store files
test_audio_files = []

# Iterate directory
for path in os.listdir(TEST_PATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(TEST_PATH, path)):
        test_audio_files.append(path)
len(test_audio_files)

7747

In [ ]:
# extract_path = '/content/input/dlsprint'
drive_path = '/content/gdrive/MyDrive/kaggleProject/dlsprint'
new_dir = ['new_train','new_validation','new_test']
for folder in new_dir:
    newFolder = os.path.join(extract_path,folder)
    # newFolder = os.path.join(drive_path,folder) 
    print(newFolder)
    if not os.path.exists(newFolder):
        os.makedirs(newFolder)

/content/gdrive/MyDrive/kaggleProject/dlsprint/new_train
/content/gdrive/MyDrive/kaggleProject/dlsprint/new_validation
/content/gdrive/MyDrive/kaggleProject/dlsprint/new_test


In [ ]:
# extract_path = '/content/input/dlsprint'
drive_path = '/content/gdrive/MyDrive/kaggleProject/dlsprint'
# New_Train_Dir = os.path.join(extract_path,'new_train') # './new_train' 
# NEW_Validation_Dir = os.path.join(extract_path,'new_validation') # './new_validation'
# New_Test_Dir = os.path.join(extract_path,'new_test') # './new_test'

# For save to drive 
New_Train_Dir = os.path.join(drive_path,'new_train') # './new_train' 
NEW_Validation_Dir = os.path.join(drive_path,'new_validation') # './new_validation'
New_Test_Dir = os.path.join(drive_path,'new_test') # './new_test'

In [ ]:
print(New_Train_Dir)

/content/gdrive/MyDrive/kaggleProject/dlsprint/new_train


## Converting the train .mp3 files into .wav and saving it inside a new directory

In [ ]:
def save_fn(filename):
    
    
    path = f"{TRAIN_PATH}/{filename}"
    save_path = f"{New_Train_Dir}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            sound = AudioSegment.from_mp3(path)
            sound = sound.set_frame_rate(16000)
            sound.export(f"{save_path}/{filename[:-4]}.wav", format="wav")
        except:
            print(path)

In [ ]:
import os, platform, subprocess, re

def get_processor_name():
    if platform.system() == "Windows":
        return platform.processor()
    elif platform.system() == "Darwin":
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin'
        command ="sysctl -n machdep.cpu.brand_string"
        return subprocess.check_output(command).strip()
    elif platform.system() == "Linux":
        command = "cat /proc/cpuinfo"
        all_info = subprocess.check_output(command, shell=True).decode().strip()
        for line in all_info.split("\n"):
            if "model name" in line:
                return re.sub( ".*model name.*:", "", line,1)
    return ""

get_processor_name()

' Intel(R) Xeon(R) CPU @ 2.20GHz'

In [ ]:
import time
start = time.time()

Parallel(n_jobs=-1, backend="multiprocessing")(
    delayed(save_fn)(filename) for filename in tqdm(train_audio_files)
)

end = time.time()
print("total time to process: {x} seconds".format(x=end-start))

  0%|          | 0/201414 [00:00<?, ?it/s]